### Disclaimer

Distribution authorized to U.S. Government agencies and their contractors. Other requests for this document shall be referred to the MIT Lincoln Laboratory Technology Office.

This material is based upon work supported by the Under Secretary of Defense for Research and Engineering under Air Force Contract No. FA8702-15-D-0001. Any opinions, findings, conclusions or recommendations expressed in this material are those of the author(s) and do not necessarily reflect the views of the Under Secretary of Defense for Research and Engineering.

© 2019 Massachusetts Institute of Technology.

The software/firmware is provided to you on an As-Is basis

Delivered to the U.S. Government with Unlimited Rights, as defined in DFARS Part 252.227-7013 or 7014 (Feb 2014). Notwithstanding any copyright notice, U.S. Government rights in this work are defined by DFARS 252.227-7013 or DFARS 252.227-7014 as detailed above. Use of this work other than as specifically authorized by the U.S. Government may violate any copyrights that exist in this work.


### Treasure Hunt Challenge

This notebook uses [Stable Baselines](https://stable-baselines.readthedocs.io/en/master/) to train an agent for the [GOSEEK-Challenge](https://github.mit.edu/TESS/goseek-challenge). 

Proximal Policy Optimization is used to train an agent defined by a CNN-LSTM network. The agent's observations consist of RGB, segmentation, and depth images and relative pose. This, along with the reward function, is defined in the [GoSeekFullPerception](https://github.mit.edu/TESS/tesse-gym/blob/master/src/tesse_gym/tasks/goseek/goseek_full_perception.py#L30) [gym environment](https://gym.openai.com/). 


__Contents__
- [Configure Environment](#Configuration)
- [Define Model](#Define-the-Model)
- [Train Model](#Train-the-Model)
- [Visualize Results](#Visualize-Results)

In [1]:
from pathlib import Path

from gym import spaces
from stable_baselines.common.policies import CnnLstmPolicy
from stable_baselines.common.vec_env import SubprocVecEnv, DummyVecEnv
from stable_baselines import PPO2
from tesse.msgs import *

from tesse_gym import get_network_config
from tesse_gym.tasks.goseek import GoSeekFullPerception, decode_observations


/home/francesco/anaconda3/envs/goseek/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/francesco/anaconda3/envs/goseek/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/francesco/anaconda3/envs/goseek/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/francesco/anaconda3/envs/goseek

# Configuration

#### Set sim path

In [2]:
filename = Path("../../goseek-challenge/simulator/goseek-v0.1.4.x86_64")
assert filename.exists(), f"Must set a valid path!"

#### Set environment parameters


__Note__ To minimize training time during initial use, we've set `total_timestamps` and `n_environments` to 1e5 and 2 respectively. Setting `total_timestamps` to 3e6 and `n_environments` to 4 should produce an agent that approximates our baseline. 

In [3]:
n_environments = 5  # number of environments to train over
total_timesteps = 500001  # number of training timesteps
scene_id = [1, 2, 3, 4, 5]  # list all available scenes
n_targets = 30  # number of targets spawned in each scene
target_found_reward = 2  # reward per found target
episode_length = 400


def make_unity_env(filename, num_env):
    """ Create a wrapped Unity environment. """

    def make_env(rank):
        def _thunk():
            env = GoSeekFullPerception(
                str(filename),
                network_config=get_network_config(worker_id=rank),
                n_targets=n_targets,
                episode_length=episode_length,
                scene_id=scene_id[rank%len(scene_id)],#np.random.choice(scene_id),
                target_found_reward=target_found_reward,
            )
            return env

        return _thunk

    return SubprocVecEnv([make_env(i) for i in range(num_env)])

#### Launch environments.

In [4]:
env = make_unity_env(filename, n_environments)

LOAD_MODEL = False

# Define the Model 

The following network assumes an observation of consisting of RGB, segmentation, and depth images along with the agent's relative pose from start. Images are processed using the Stable Baseline default CNN. The resulting feature vector is concatenated with the pose vector and given to an LSTM.

In [5]:
import tensorflow as tf
from stable_baselines.common.policies import nature_cnn

#### Define network to consume images and pose

In [6]:

def decode_tensor_observations(observation, img_shape=(-1, 240, 320, 5)):
    """ Decode observation vector into images and poses.

    Args:
        observation (np.ndarray): Shape (N,) observation array of flattened
            images concatenated with a pose vector. Thus, N is equal to N*H*W*C + N*3.
        img_shape (Tuple[int, int, int, int]): Shapes of all images stacked in (N, H, W, C).
            Default value is (-1, 240, 320, 5).
    
    Returns:
        Tuple[tf.Tensor, tf.Tensor]: Tensors with the following information
            - Tensor of shape (N, `img_shape[1:]`) containing RGB,
                segmentation, and depth images stacked across the channel dimension.
            - Tensor of shape (N, 3) containing (x, y, heading) relative to starting point.
                (x, y) are in meters, heading is given in degrees in the range [-180, 180].
    """
    
    imgs = tf.reshape(observation[:, :-3], img_shape)[..., -2:]
    pose = observation[:, -3:]
#     im1 = tf.image.resize(
#         imgs[..., :3], tf.constant([img_shape[1]//10, img_shape[2]//10], dtype=np.int32), method=tf.image.ResizeMethod.NEAREST_NEIGHBOR
#     )
#     im2 = tf.image.resize(
#         tf.expand_dims(imgs[..., 3], axis=3), tf.constant([img_shape[1]//10, img_shape[2]//10], dtype=np.int32), method=tf.image.ResizeMethod.NEAREST_NEIGHBOR
#     )
#     im3 = tf.image.resize(
#         tf.expand_dims(imgs[..., 4], axis=3), tf.constant([img_shape[1]//10, img_shape[2]//10], dtype=np.int32), method=tf.image.ResizeMethod.NEAREST_NEIGHBOR
#     )

#     new_imgs = im2 #tf.concat([im1, im2, im3], axis=3)


#     return tf.reshape(new_imgs, [-1, new_imgs.shape[1]*new_imgs.shape[2]*new_imgs.shape[3]]), imgs, pose
    return imgs, pose

In [7]:

from stable_baselines.a2c.utils import conv, linear, conv_to_fc

def attention_cnn(scaled_images, **kwargs):
    """Nature CNN with region-sensitive module"""
        
    def linear2d(input_tensor, num_hidden, scope):
        b, h, w = input_tensor.shape
        with tf.variable_scope(scope):
            tensors = []
            for i in range(h):
                weight = tf.get_variable("w"+str(i), [w, num_hidden], initializer=tf.initializers.orthogonal())
                bias = tf.get_variable("b"+str(i), [num_hidden], initializer=tf.constant_initializer(0.0))
                tensors.append(tf.matmul(input_tensor[:,i,:], weight) + bias)
            return tf.stack(tensors, axis=1)
        
    def attention_block(tensor, g, scope):
        b, h, w, f = tensor.shape
        ls = tf.reshape(tensor, (-1, h*w, f))
        print("ls",ls.get_shape())
        g_size = g.get_shape()[-1].value
        print("g", g.get_shape())

        with tf.variable_scope(scope):
            lsat = linear2d(ls, num_hidden=g_size, scope='lsat') # (-1, h*w, g_size)
            lsat = tf.nn.relu(lsat)
            print("lsat", lsat.get_shape())
            ### TODO is including also the batch dimension correct? ###
            g_tiled = tf.tile(tf.reshape(g, (-1, 1, g_size)), [1, h*w, 1])
            compatibility = tf.reduce_sum(tf.multiply(lsat, g_tiled), axis=-1, keepdims=True) #tf.tensordot(lsat, g_tiled, axes=((-1), (-1))) # (-1, h*w, 1)
    #         compatibility = tf.reshape(compatibility, shape=[-1, h*w, 1]) # (-1, h*w)
            print("compatibility", compatibility.get_shape())
            attention = tf.nn.softmax(compatibility, axis=1, name="attention_softmax") # (-1, h*w)
        #     attention = tf.tile(tf.reshape(attention, shape=(-1, h*w, 1)), [1, 1, f]) # (-1, h*w, f)
            attention_tiled = tf.tile(attention, [1, 1, f]) # (-1, h*w, f)
            print("attention", attention_tiled.get_shape())
            weighted_ls = attention_tiled * ls
            return weighted_ls, attention


    c1 = tf.nn.relu(conv(scaled_images, 'c1', n_filters=16, filter_size=8, stride=2, init_scale=np.sqrt(2), **kwargs))
    c2 = tf.nn.relu(conv(c1, 'c2', n_filters=24, filter_size=4, stride=2, init_scale=np.sqrt(2), **kwargs))
    c3 = tf.nn.relu(conv(c2, 'c3', n_filters=32, filter_size=3, stride=1, init_scale=np.sqrt(2), **kwargs))
#     c3 = tf.nn.l2_normalize(c3, axis=-1)

    g = tf.nn.relu(linear(conv_to_fc(c3), n_hidden=64, scope="g", init_scale=np.sqrt(2)))
    
#     g2, attn2_layer = attention_block(c2, g, 'attn2') 
#     g2 = conv_to_fc(g2)
    g3, attn3_layer = attention_block(c3, g, 'attn3') 
    g3 = conv_to_fc(g3)
    
    gsa = g3#tf.concat((g2, g3), 1)
    
    lastln = tf.nn.relu(linear(gsa, 'fc1', n_hidden=256, init_scale=np.sqrt(2)))
    
    return lastln 

In [8]:
def image_and_pose_network(observation, **kwargs):
    """ Network to process image and pose data.
    
    Use the stable baselines nature_cnn to process images. The resulting
    feature vector is then combined with the pose estimate and given to an
    LSTM (LSTM defined in PPO2 below).
    
    Args:
        raw_observations (tf.Tensor): 1D tensor containing image and 
            pose data.
        
    Returns:
        tf.Tensor: Feature vector. 
    """
    orig_imgs, pose = decode_tensor_observations(observation)
    scaled_imgs = tf.image.resize_images(orig_imgs, [40, 40], method=1) # 1: nearest
    image_features = attention_cnn(scaled_imgs)
#     print(image_features.shape, imgs.shape, pose.shape)
    return tf.concat((image_features, pose), axis=-1)

#### Register custom network

Outputs of the network defined above will be fed into an LSTM defined below in PPO2.

In [9]:
if tf.test.gpu_device_name(): 
    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

Default GPU Device:/device:GPU:0


In [10]:
# policy_kwargs = {'cnn_extractor': image_and_pose_network}
policy_kwargs = {'cnn_extractor': image_and_pose_network}

In [11]:

if LOAD_MODEL:
    MODEL_WEIGHTS_PATH = "results/goseek-ppo-realattention-forwardreward-fartargets/final_model.pkl"
    assert MODEL_WEIGHTS_PATH, f"Must give a model weights path!"
    model = PPO2.load(str(MODEL_WEIGHTS_PATH),env=env, tensorboard_log="./tensorboard/", gamma=0.995, learning_rate=0.0002)
else:
    model = PPO2(
        CnnLstmPolicy,
        env,
        n_steps=100,
        verbose=1,
        tensorboard_log="./tensorboard/",
        nminibatches=5,
        gamma=0.995,
        learning_rate=0.00025,
        policy_kwargs=policy_kwargs,
    )

Instructions for updating:
Colocations handled automatically by placer.
ls (5, 25, 32)
g (5, 64)
lsat (5, 25, 64)
compatibility (5, 25, 1)
attention (5, 25, 32)
ls (100, 25, 32)
g (100, 64)
lsat (100, 25, 64)
compatibility (100, 25, 1)
attention (100, 25, 32)
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


# Train the Model

#### Define logging directory and callback function to save checkpoints

In [12]:
log_dir = Path("results/goseek-ppo-realattention-forwardreward-50fartargets")
log_dir.mkdir(parents=True, exist_ok=True)

total_updates = 0
def save_checkpoint_callback(local_vars, global_vars):
    global total_updates
#     print(f"=== local vars ===\n{local_vars.keys()}")  # add this line 
#     total_updates = local_vars["n_updates"]
    total_updates += 1
    if total_updates % 1000 == 0:
        local_vars["self"].save(str(log_dir / f"{total_updates:06d}.pkl"))
        print("Saving model")

In [13]:
if LOAD_MODEL:
    model.learn(total_timesteps=total_timesteps, tb_log_name='50fartargets', callback=save_checkpoint_callback, reset_num_timesteps=False)
else:
    model.learn(total_timesteps=total_timesteps, callback=save_checkpoint_callback)

## TODO:
## add little penalty (-0.01) for using the grab action... because it grabs too much
## DONE add little reward (0.01) for going forward... bcs it rotates too much
## visualize attention?

--------------------------------------
| approxkl           | 0.0010325776  |
| clipfrac           | 0.0           |
| explained_variance | -0.0137       |
| fps                | 8             |
| n_updates          | 1             |
| policy_entropy     | 1.3843722     |
| policy_loss        | -0.0061120847 |
| serial_timesteps   | 100           |
| time_elapsed       | 7.01e-05      |
| total_timesteps    | 500           |
| value_loss         | 0.032731924   |
--------------------------------------
-------------------------------------
| approxkl           | 0.0023111817 |
| clipfrac           | 0.0125       |
| explained_variance | 0.278        |
| fps                | 15           |
| n_updates          | 2            |
| policy_entropy     | 1.370815     |
| policy_loss        | -0.002890921 |
| serial_timesteps   | 200          |
| time_elapsed       | 58           |
| total_timesteps    | 1000         |
| value_loss         | 0.022559594  |
-------------------------------------

-------------------------------------
| approxkl           | 0.0064087496 |
| clipfrac           | 0.0995       |
| explained_variance | 0.136        |
| fps                | 17           |
| n_updates          | 18           |
| policy_entropy     | 0.7588175    |
| policy_loss        | -0.008281929 |
| serial_timesteps   | 1800         |
| time_elapsed       | 484          |
| total_timesteps    | 9000         |
| value_loss         | 0.096004926  |
-------------------------------------
-------------------------------------
| approxkl           | 0.015476751  |
| clipfrac           | 0.16199997   |
| explained_variance | 0.554        |
| fps                | 18           |
| n_updates          | 19           |
| policy_entropy     | 0.6355119    |
| policy_loss        | 0.0031921086 |
| serial_timesteps   | 1900         |
| time_elapsed       | 513          |
| total_timesteps    | 9500         |
| value_loss         | 0.039673887  |
-------------------------------------
Saving model

--------------------------------------
| approxkl           | 0.00017743678 |
| clipfrac           | 0.0015        |
| explained_variance | 0.976         |
| fps                | 20            |
| n_updates          | 35            |
| policy_entropy     | 0.02910234    |
| policy_loss        | -0.0006334331 |
| serial_timesteps   | 3500          |
| time_elapsed       | 908           |
| total_timesteps    | 17500         |
| value_loss         | 0.00016268584 |
--------------------------------------
-------------------------------------
| approxkl           | 0.0010090873 |
| clipfrac           | 0.003        |
| explained_variance | 0.981        |
| fps                | 19           |
| n_updates          | 36           |
| policy_entropy     | 0.047466487  |
| policy_loss        | -0.001282389 |
| serial_timesteps   | 3600         |
| time_elapsed       | 932          |
| total_timesteps    | 18000        |
| value_loss         | 4.873485e-05 |
-------------------------------------

-------------------------------------
| approxkl           | 0.0011158899 |
| clipfrac           | 0.0235       |
| explained_variance | 0.0353       |
| fps                | 20           |
| n_updates          | 52           |
| policy_entropy     | 0.17394587   |
| policy_loss        | -0.003653658 |
| serial_timesteps   | 5200         |
| time_elapsed       | 1.32e+03     |
| total_timesteps    | 26000        |
| value_loss         | 0.18828496   |
-------------------------------------
----------------------------------------
| approxkl           | 3.445618e-05    |
| clipfrac           | 0.0             |
| explained_variance | 0.129           |
| fps                | 19              |
| n_updates          | 53              |
| policy_entropy     | 0.19568841      |
| policy_loss        | -0.000103112754 |
| serial_timesteps   | 5300            |
| time_elapsed       | 1.35e+03        |
| total_timesteps    | 26500           |
| value_loss         | 0.07528128      |
--------------

--------------------------------------
| approxkl           | 0.014897658   |
| clipfrac           | 0.033999998   |
| explained_variance | 0.435         |
| fps                | 26            |
| n_updates          | 69            |
| policy_entropy     | 0.19078429    |
| policy_loss        | -7.811752e-05 |
| serial_timesteps   | 6900          |
| time_elapsed       | 1.74e+03      |
| total_timesteps    | 34500         |
| value_loss         | 0.057495814   |
--------------------------------------
Saving model
--------------------------------------
| approxkl           | 0.0030762542  |
| clipfrac           | 0.019499999   |
| explained_variance | 0.99          |
| fps                | 25            |
| n_updates          | 70            |
| policy_entropy     | 0.12756538    |
| policy_loss        | -0.0018031945 |
| serial_timesteps   | 7000          |
| time_elapsed       | 1.76e+03      |
| total_timesteps    | 35000         |
| value_loss         | 0.0004824095  |
------------

...

--------------------------------------
| approxkl           | 0.0010499273  |
| clipfrac           | 0.01          |
| explained_variance | 0.965         |
| fps                | 25            |
| n_updates          | 934           |
| policy_entropy     | 0.17051837    |
| policy_loss        | -0.0023886126 |
| serial_timesteps   | 93400         |
| time_elapsed       | 2.29e+04      |
| total_timesteps    | 467000        |
| value_loss         | 0.0012784199  |
--------------------------------------
--------------------------------------
| approxkl           | 0.010029952   |
| clipfrac           | 0.094         |
| explained_variance | 0.456         |
| fps                | 22            |
| n_updates          | 935           |
| policy_entropy     | 0.5411286     |
| policy_loss        | -0.0049098767 |
| serial_timesteps   | 93500         |
| time_elapsed       | 2.29e+04      |
| total_timesteps    | 467500        |
| value_loss         | 0.50176615    |
-------------------------

--------------------------------------
| approxkl           | 0.023682943   |
| clipfrac           | 0.083         |
| explained_variance | 0.097         |
| fps                | 20            |
| n_updates          | 951           |
| policy_entropy     | 0.34489673    |
| policy_loss        | -0.0065740338 |
| serial_timesteps   | 95100         |
| time_elapsed       | 2.33e+04      |
| total_timesteps    | 475500        |
| value_loss         | 0.08176492    |
--------------------------------------
-------------------------------------
| approxkl           | 0.0021764874 |
| clipfrac           | 0.0215       |
| explained_variance | 0.735        |
| fps                | 22           |
| n_updates          | 952          |
| policy_entropy     | 0.20321998   |
| policy_loss        | -0.005298433 |
| serial_timesteps   | 95200        |
| time_elapsed       | 2.33e+04     |
| total_timesteps    | 476000       |
| value_loss         | 0.0016041184 |
-------------------------------------

--------------------------------------
| approxkl           | 0.007992472   |
| clipfrac           | 0.085999995   |
| explained_variance | 0.155         |
| fps                | 18            |
| n_updates          | 968           |
| policy_entropy     | 0.71762145    |
| policy_loss        | -0.0062879412 |
| serial_timesteps   | 96800         |
| time_elapsed       | 2.37e+04      |
| total_timesteps    | 484000        |
| value_loss         | 0.55290496    |
--------------------------------------
-------------------------------------
| approxkl           | 0.004245664  |
| clipfrac           | 0.058499992  |
| explained_variance | -0.0836      |
| fps                | 20           |
| n_updates          | 969          |
| policy_entropy     | 0.46703792   |
| policy_loss        | -0.003601852 |
| serial_timesteps   | 96900        |
| time_elapsed       | 2.37e+04     |
| total_timesteps    | 484500       |
| value_loss         | 0.13017297   |
-------------------------------------

--------------------------------------
| approxkl           | 0.0008786831  |
| clipfrac           | 0.013499999   |
| explained_variance | 0.439         |
| fps                | 28            |
| n_updates          | 985           |
| policy_entropy     | 0.21047473    |
| policy_loss        | -0.0027629987 |
| serial_timesteps   | 98500         |
| time_elapsed       | 2.41e+04      |
| total_timesteps    | 492500        |
| value_loss         | 0.028892815   |
--------------------------------------
--------------------------------------
| approxkl           | 0.0011054     |
| clipfrac           | 0.013499999   |
| explained_variance | 0.0516        |
| fps                | 20            |
| n_updates          | 986           |
| policy_entropy     | 0.2664786     |
| policy_loss        | -0.0021086163 |
| serial_timesteps   | 98600         |
| time_elapsed       | 2.41e+04      |
| total_timesteps    | 493000        |
| value_loss         | 0.42919302    |
-------------------------

KeyboardInterrupt: 

In [ ]:
model.save(str(log_dir) +"/final_model.pkl")

# Visualize Results

__Note__: Stable-Baselines requires that policy input dimensions be consistent across training and testing. Thus, the number of environments used for visualization must be a multiple of the number of environments used for training. The observation vector is then appropriately duplicated during inference. 

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt

#### Load model

In [ ]:
MODEL_WEIGHTS_PATH = "results/goseek-ppo-realattention-forwardreward-50fartargets/final_model.pkl"
assert MODEL_WEIGHTS_PATH, f"Must give a model weights path!"

model = PPO2.load(str(MODEL_WEIGHTS_PATH))
n_train_envs = model.act_model.initial_state.shape[0]

#### Visualize all observed images

In [ ]:
obs = env.reset()
img_shape = (-1, 240, 320, 5)
imgs = np.reshape(obs[:, :-3], img_shape)[..., -2:]
print(imgs.shape)
rgb, segmentation, depth, pose = decode_observations(obs)
lstm_state = None

print(pose)

assert (
    n_train_envs % obs.shape[0] == 0
), f"The number of visualization environments must be a multiple of the training environments"

In [ ]:
fig, ax = plt.subplots(1, 3)
ax[0].imshow(rgb[0])
ax[1].imshow(segmentation[0])
ax[2].imshow(depth[0])

print(rgb[0].shape)
print(segmentation[0].shape)
print(depth[0].shape)

#### Run an episode and plot the first person agent view

In [ ]:
import cv2

# TODO:
# - check that the state is correct (segmentation and values of classes)
# - check the robot pose values
done = False
fig, ax = plt.subplots(1, obs.shape[0])
ax = [ax] if obs.shape[0] == 1 else ax

for i in range(episode_length):
    actions, lstm_state = model.predict(
        np.concatenate((n_train_envs // obs.shape[0]) * [obs]),
        state=lstm_state,
        deterministic=False
    )

    actions = actions[: obs.shape[0]]

    obs, reward, done, _ = env.step(actions)
#     print(actions, done, reward)

    plt.cla()
    rgb, segmentation, depth, pose = decode_observations(obs)
    

    for i in range(obs.shape[0]):
        print(reward)
#         print(segmentation[i][:,10])
        print(np.max(depth[i]), np.min(depth[i]))
        ax[i].imshow(cv2.resize(depth[i], dsize=(40,40), interpolation=cv2.INTER_NEAREST), vmin=0., vmax=1.)
#         ax[i].imshow(depth[i], vmin=0., vmax=1.)

    
    fig.canvas.draw()

obs = env.reset()
rgb, segmentation, depth, pose = decode_observations(obs)
lstm_state = None